# Markov Chain Monte Carlo for Tree structures
## A software package to calculate statistic problems including (phylogenetic) tree structures

Typical data format in phylogenetic linguistics

|Language | Mountain | You | ...
|---------|------|-----|-----
|Swedish  | 1 |  1  | ...
|Norwegian| 1 |  1  | ...
|Italian  | 2 |  1  | ...

Such data is used for phylogenetic inference using Bayesian methods. Based on an Markov Process wich describes the untderlying evolutionary process of character evolution Markov Chain Monte Carlo Methods can be used to estimate a posterior of phylogenetic trees. Using such (or other) trees, further statistical questions can be asked. Several of these questions are based on a statistical model whose parameters need to be infered using Markov Chain Monte Carlo methods. This requires a flexible framework which can be used to define these models. Additionally, inference should be relatively fast.

The Julia programming language is a new language which aims to be high performance and easy to write. This makes it a good starting point to develop a system which can be used by many to define models for their needs but also efficient in calculating these models. 

The [MCPhylo package](https://MCPhylojl.readthedocs.io/en/latest/intro.html) offers a good starting point to develop an infrastructure for said tasks using the Julia Programming Language.

In [1]:
include("../MCPhylo/src/MCPhylo.jl");
using .MCPhylo;
using Random;
Random.seed!(1234);

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /home/johannes/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


As a first step read in the data from the data file. This function creates a tree object (`m_tree`) which is a random binary tree with the languages specified in the data fiel as the leaves and an array object, which stores the character information.

In [2]:
m_tree, df = make_tree_with_data("../local/development.nex"); # load your own nexus file

In a next step all the relevant information needs to be stored in a dictionary so they can be used later on. The entries `:nnodes`, `:nsites` and `:nbase` store the the dimensions of the data array. Additionally the data array is log transformed. The log transformation is necessary for the likelihood computation.

In [3]:
my_data = Dict{Symbol, Any}(
  :mtree => m_tree,
  :df => log.(df),
  :nnodes => size(df)[1],
  :nbase => size(df)[2],
  :nsites => size(df)[3])


Dict{Symbol,Any} with 5 entries:
  :df     => [0.0 -Inf; 0.0 -Inf; … ; -Inf -Inf; -Inf -Inf]…
  :nnodes => 17
  :mtree  => "17"
  :nsites => 3132
  :nbase  => 2

The important part is the model definition. The idea is to define a model in terms of a graph. The graph represents the model and thus the explicit relationships between the parameters. Parameters can be specified as different nodes in the graph. There are *Stochastic*, *Logical* and *Input* nodes.

* Stochastic nodes represent variables which have a prior or likelihood specification associated with them.
* Logical nodes are deterministic functions of other nodes. 
* Input nodes are fixed model terms, which are fixed in your analysis. (They are not explicitly specified in the model definition.)

Nodes in the model graph are specified by three elements.

1. The dimension of the node, which is either nothing (a scalar), an integer (an array of this dimension) or the Node() identifier to identify a Tree type node.
2. A function indicating the distribution associated with this node, or a deterministic function for logical nodes.
3. A boolean identifying if the value of this node should be monitored.

In [4]:
# model setup
model =  Model(
    df = Stochastic(3,
    (mtree, mypi, rates, nnodes, nbase, nsites) -> PhyloDist(mtree, mypi, rates, nnodes, nbase, nsites), false
    ),
    mypi = Stochastic( () -> Uniform(0.0,1.0)),
    mtree = Stochastic(Node(), () -> CompoundDirichlet(1.0,1.0,0.100,1.0), true),
    rates = Logical(1,(mymap, av) -> [av[convert(UInt8,i)] for i in mymap],false),
    mymap = Stochastic(1,() -> Categorical([0.25, 0.25, 0.25, 0.25]), false),
    av = Stochastic(1,() -> Dirichlet([1.0, 1.0, 1.0, 1.0]))
     );


The model graph can plotted to verify the model specification.

In [ ]:
draw(model, filename="my_graph.dot");

This command saves the graph structure into a dot file which. 

![g](g.png)

The next important step is to specify the sampling scheme. The sampling scheme defines how the different values of the stochastic nodes are sampled. The original MCPhylo package provides several samplers for scalar and array valued stochastic nodes. The `:mypi` and `:av` parameters are sampled using samplers provided by the MCPhylo package. (See [Here](https://MCPhylojl.readthedocs.io/en/latest/samplers.html) for a list of available MCPhylo samplers) For the `:mymap` feature a discrete random walk metropolis sampler is added. The tree is sampled using a Probabilistic Path Hamiltonian Monte Carlo sampler ([Dinh et. al. 2017](https://arxiv.org/pdf/1702.07814.pdf)).

 

In [5]:
scheme = [ProbPathHMC(:mtree, 3.0,0.02, 0.001, :provided),
          Slice(:mypi, 0.05, Univariate),
          SliceSimplex(:av, scale=0.02),
          RWMC(:mymap)
          ];
setsamplers!(model, scheme);

The Probabilistic Path Hamiltonian Monte Carlo sampler defines the hamiltonian dynamics for the space of tree structures. By using this method trees can be sampled more efficiently and the tree space can be explored faster.

As a final step the intial (random) values of the model need to be specified.

In [6]:
# intial model values
inits = [ Dict(
    :mtree => my_data[:mtree],
    :mypi=> 0.5,
    :df => my_data[:df],
    :nnodes => size(my_data[:nnodes]),
    :nbase => size(my_data[:nbase]),
    :nsites => size(my_data[:nsites]),
    :mymap=>rand(Categorical([0.25, 0.25, 0.25, 0.25]),my_data[:nsites]),
    :av => rand(Dirichlet([0.25, 0.25, 0.25, 0.25]))
    )
    ];

The actual parameter estimation of the model parameters is done by the `mcmc` function. This function assembles all the elements specified above. In essence it takes the same parameters as the original `mcmc` functionf from the MCPhylo package. It is just extendend by the `trees` parameter. This parameter accepts a boolean value. If set to `true` the tree structures are monitored. Otherwise the tree structures are *lost* and can not be inspected afterwards.

In [ ]:
sim = mcmc(model, my_data, inits, 10, burnin=2,thin=1, chains=1, trees=true)

The MCPhylo package offers some diagnostics for MCMC runs. They can all be applied to the the output of the MCMC run. Additionally there now exists a function `to_file` which flushes all monitored parameters and the collection of trees (if they are monitored) to a files which can be read by the [*Tracer Program*](http://tree.bio.ed.ac.uk/software/tracer/). The second parameter of the `to_file` function specifies the path where the resulting file(s) should be stored.

In [ ]:
to_file(sim, "")

## Some available Distributions for trees

Prior distributions over Tree structures

* Compound Dirichlet ([Zhang, Rannala and Yang 2012.](https://doi.org/10.1093/sysbio/sys030))
* Strict Molecular Clock - Birth Death ([Yang & Rannala 1997](https://doi.org/10.1093/oxfordjournals.molbev.a025811))
* Strict Molecular Clock - Simplified Birth Death ([Yang & Rannala 1996](https://doi.org/10.1007/BF02338839))
